In [56]:
import cv2
import numpy as np
import os
from lines import *
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Adjust the path as per your Tesseract installation

In [57]:
image = cv2.imread('cropped/AA AT-096-[0].jpg', 0)  # 0 loads the image in grayscale

In [58]:
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return (x, y)

In [59]:
def transform_image(image, points):
    # Convert the input points to a NumPy array
    points = np.array(points, dtype=np.float32)

    # Calculate the bounding box for the source region
    min_x, min_y = np.min(points, axis=0)
    max_x, max_y = np.max(points, axis=0)

    # Calculate the aspect ratio of the source region
    aspect_ratio = (max_x - min_x) / (max_y - min_y)

    # Define the target rectangle based on the aspect ratio
    target_height = 300  # Adjust this value based on your preference
    target_width = int(target_height * aspect_ratio)

    target_rect = np.array([[0, 0], [target_width, 0], [target_width, target_height], [0, target_height]], dtype=np.float32)

    # Calculate the perspective transformation matrix
    matrix = cv2.getPerspectiveTransform(points, target_rect)

    # Apply the perspective transformation
    result = cv2.warpPerspective(image, matrix, (target_width, target_height))

    return result

In [60]:
def detect_text(binary_image):
    # Specify the OCR engine mode
    custom_config = r'--oem 3 --psm 6'

    # Perform OCR on the preprocessed image
    text = pytesseract.image_to_string(binary_image, config=custom_config)
    print("OCR Output:", text)

In [61]:
# Get the bounding lines and show them on the image. Do this for all images in the folder
for filename in os.listdir('cropped'):
    image = cv2.imread('cropped/' + filename, 0)
    h, v = get_bounding_lines(image, 15, 70, 60, 32)
    
    if h is None or v is None:
        print(filename + '  :  failed to find bounding lines')
        continue

    # Show the lines on the image
    image1 = image.copy()
    cv2.line(image1, (h[0], h[1]), (h[2], h[3]), 255, 2)
    cv2.line(image1, (h[4], h[5]), (h[6], h[7]), 255, 2)
    cv2.line(image1, (v[0], v[1]), (v[2], v[3]), 255, 2)
    cv2.line(image1, (v[4], v[5]), (v[6], v[7]), 255, 2)
    
    # Get the intersection points
    p1 = line_intersection(((h[0], h[1]), (h[2], h[3])), ((v[0], v[1]), (v[2], v[3])))
    p2 = line_intersection(((h[0], h[1]), (h[2], h[3])), ((v[4], v[5]), (v[6], v[7])))
    p3 = line_intersection(((h[4], h[5]), (h[6], h[7])), ((v[4], v[5]), (v[6], v[7])))
    p4 = line_intersection(((h[4], h[5]), (h[6], h[7])), ((v[0], v[1]), (v[2], v[3])))

    image2 = image.copy()
    # Show the points on the image
    cv2.circle(image2, (int(p1[0]), int(p1[1])), 3, 255, -1)
    cv2.circle(image2, (int(p2[0]), int(p2[1])), 3, 255, -1)
    cv2.circle(image2, (int(p3[0]), int(p3[1])), 3, 255, -1)
    cv2.circle(image2, (int(p4[0]), int(p4[1])), 3, 255, -1)

    # Get the rectangle image
    image3 = transform_image(image, [p1, p2, p3, p4])
    
    #scale all 3 images to a fixed width
    width = 300
    image1 = cv2.resize(image1, (width, int(image1.shape[0] * width / image1.shape[1])))
    image2 = cv2.resize(image2, (width, int(image2.shape[0] * width / image2.shape[1])))
    image3 = cv2.resize(image3, (width, int(image3.shape[0] * width / image3.shape[1])))

    # Make all images rgb
    image1 = cv2.cvtColor(image1, cv2.COLOR_GRAY2RGB)
    image2 = cv2.cvtColor(image2, cv2.COLOR_GRAY2RGB)
    image3 = cv2.cvtColor(image3, cv2.COLOR_GRAY2RGB)

    # Insert a red line at the bottom of each image
    image1 = cv2.line(image1, (0, image1.shape[0] - 1), (image1.shape[1] - 1, image1.shape[0] - 1), (0, 0, 255), 2)
    image2 = cv2.line(image2, (0, image2.shape[0] - 1), (image2.shape[1] - 1, image2.shape[0] - 1), (0, 0, 255), 2)
    image3 = cv2.line(image3, (0, image3.shape[0] - 1), (image3.shape[1] - 1, image3.shape[0] - 1), (0, 0, 255), 2)

    # Perform OCR on the rectangle image
    detect_text(image3)

    # Show all three images in one line as the output of the cell
    concatenated = np.concatenate((image1, image2, image3), axis=0)
    cv2.imshow('All', concatenated)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

OCR Output: ASAA-017

OCR Output: AMAA 302

OCR Output: AMSAA-489

OCR Output: 7
Mi

OCR Output: AASAB-264

OCR Output: AA®AB-297

OCR Output: ANMAB-74)

OCR Output: AN AB-767

OCR Output: AN AB-767

OCR Output: ASAB-949

OCR Output: 
OCR Output: AeAD-040

OCR Output: ANAD-336

OCR Output: ASAN.27,

OCR Output: AASAD-374

OCR Output: AASAD-446

OCR Output: ANSAD-456

OCR Output: ——S—t—ts

OCR Output: AASAD-468

OCR Output: ‘Va
NG.



KeyboardInterrupt: 